In [104]:
import os
import requests
import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv

load_dotenv()

True

In [105]:
token = os.environ.get("AVIATIONSTACK_TOKEN")

params = {
  'access_key': token
}

In [207]:
base_url = "http://api.aviationstack.com/v1"

def flatten(l):
    return [item for sublist in l for item in sublist]

def get_flights(limit=100, max_=1000):
    endpoint = "flights"
    url = os.path.join(base_url, endpoint)
    
    params["dep_iata"] = "SCL"
    
    for offset in tqdm(range(0, max_, limit)):
        params["limit"] = limit
        params["offset"] = offset + 1
    
        yield requests.get(url, params).json()["data"]

In [ ]:
flights = flatten(get_flights())

In [147]:
def parse_departures(flights):
    for flight in flights:       
        departure = flight["departure"]
        
        yield {
            "flight_status": flight["flight_status"],
            "flight_date": flight["flight_date"],
            "flight_number": flight["flight"]["iata"],
            "airline_iata": flight["airline"]["iata"],
            **departure,
        }

def parse_arrivals(flights):
    for flight in flights:
        arrival = flight["arrival"]
        
        yield {
            "flight_number": flight["flight"]["iata"],
            **arrival,
        }

In [148]:
df_departures = pd.DataFrame(parse_departures(flights))
df_arrivals = pd.DataFrame(parse_arrivals(flights))

In [149]:
df = pd.merge(
    df_departures,
    df_arrivals,
    on="flight_number",
    suffixes=("_departure", "_arrival")
)

In [316]:
columns = [
    "flight_date",
    "flight_number",
    "flight_status",
    "airline_iata",
    
    "iata_departure",
    "scheduled_departure",
    "actual_departure",

    "iata_arrival",
    "scheduled_arrival",
    "actual_arrival",
    
]

dataset = df[columns].copy()

dataset["flight_date"] = pd.to_datetime(dataset["flight_date"])
dataset["scheduled_departure"] = pd.to_datetime(dataset["scheduled_departure"])
dataset["actual_departure"] = pd.to_datetime(dataset["actual_departure"])
dataset["scheduled_arrival"] = pd.to_datetime(dataset["scheduled_arrival"])
dataset["actual_arrival"] = pd.to_datetime(dataset["actual_arrival"])

dataset["delay_departure"] = (dataset["actual_departure"] - dataset["scheduled_departure"]).dt.seconds
dataset["delay_arrival"] = (dataset["actual_arrival"] - dataset["scheduled_arrival"]).dt.seconds

# Get only landed flights
dataset = dataset.query("flight_status=='landed'")

dataset["flight_day"] = dataset["flight_date"].dt.day
dataset["flight_month"] = dataset["flight_date"].dt.month
dataset["flight_year"] = dataset["flight_date"].dt.year

del dataset["flight_date"]
del dataset["flight_status"]

dataset = dataset.dropna()

# Assumption: is a delayed if delay >= 25 minutes

dataset["delayed_departure"] = dataset["delay_departure"] >= 60 * 25
dataset["delayed_arrival"] = dataset["delay_arrival"] >= 60 * 25
dataset

,flight_number,airline_iata,iata_departure,scheduled_departure,actual_departure,iata_arrival,scheduled_arrival,actual_arrival,delay_departure,delay_arrival,flight_day,flight_month,flight_year,delayed_departure,delayed_arrival
0,AM7513,AM,SCL,2023-06-22 13:20:00+00:00,2023-06-22 14:18:00+00:00,AEP,2023-06-22 16:20:00+00:00,2023-06-22 16:49:00+00:00,3480.0,1740.0,22,6,2023,True,True
1,AM7513,AM,SCL,2023-06-22 13:20:00+00:00,2023-06-22 14:18:00+00:00,AEP,2023-06-21 16:20:00+00:00,2023-06-21 16:17:00+00:00,3480.0,86220.0,22,6,2023,True,True
2,AM7513,AM,SCL,2023-06-21 13:20:00+00:00,2023-06-21 13:37:00+00:00,AEP,2023-06-22 16:20:00+00:00,2023-06-22 16:49:00+00:00,1020.0,1740.0,21,6,2023,False,True
3,AM7513,AM,SCL,2023-06-21 13:20:00+00:00,2023-06-21 13:37:00+00:00,AEP,2023-06-21 16:20:00+00:00,2023-06-21 16:17:00+00:00,1020.0,86220.0,21,6,2023,False,True
7,AV116,AV,SCL,2023-06-21 15:05:00+00:00,2023-06-21 15:20:00+00:00,BOG,2023-06-21 20:25:00+00:00,2023-06-21 19:53:00+00:00,900.0,84480.0,21,6,2023,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,H21143,H2,SCL,2023-06-21 08:51:00+00:00,2023-06-21 08:58:00+00:00,ZCO,2023-06-21 10:15:00+00:00,2023-06-21 10:02:00+00:00,420.0,85620.0,21,6,2023,False,True
962,IB7082,IB,SCL,2023-06-21 12:48:00+00:00,2023-06-21 12:57:00+00:00,ZCO,2023-06-21 14:14:00+00:00,2023-06-21 14:03:00+00:00,540.0,85740.0,21,6,2023,False,True
963,JA355,JA,SCL,2023-06-21 07:51:00+00:00,2023-06-21 08:21:00+00:00,ZCO,2023-06-21 09:15:00+00:00,2023-06-21 09:26:00+00:00,1800.0,660.0,21,6,2023,True,False
965,LA33,LA,SCL,2023-06-21 16:08:00+00:00,2023-06-21 16:21:00+00:00,ZCO,2023-06-21 17:32:00+00:00,2023-06-21 17:22:00+00:00,780.0,85800.0,21,6,2023,False,True


In [317]:
dataset.to_parquet("../data/flights.parquet", index=False)